# Importing Libraries

In [2]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sys
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize 

# Importing Dataset

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ibm_api_key_id',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'Bucket'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


# Unzipping Dataset

In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)
    

In [5]:
import os
os.listdir('.')

['.virtual_documents', 'body.h5', 'body.tgz', 'Dataset', 'body_cloud.tar.gz']

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.1,zoom_range=0.1,horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# MODEL FOR BODY TYPE DETECTION

In [28]:
trainPath = '/home/wsuser/work/Dataset/body/training'
testPath = '/home/wsuser/work/Dataset/body/validation'

In [29]:
training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [30]:
training_set.class_indices

{'00-front': 0, '01-rear': 1, '02-side': 2}

# Declaring Model Variable

In [31]:
vgg=VGG16(input_shape=(244,244,3),weights='imagenet',include_top=False)



for layer in vgg.layers:
  layer.trainable=False


x=Flatten()(vgg.output)


prediction=Dense(3,activation='softmax')(x)


model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0   

# Compiling the Model

In [32]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

# Training model

In [33]:
r = model.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

/tmp/wsuser/ipykernel_164/289406290.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
97/97 [==============================] - 339s 3s/step - loss: 1.1511 - acc: 0.5459 - val_loss: 0.9324 - val_acc: 0.6294
Epoch 2/25
97/97 [==============================] - 328s 3s/step - loss: 0.6237 - acc: 0.7534 - val_loss: 0.7954 - val_acc: 0.6941
Epoch 3/25
97/97 [==============================] - 331s 3s/step - loss: 0.4937 - acc: 0.8070 - val_loss: 1.1732 - val_acc: 0.6176
Epoch 4/25
97/97 [==============================] - 326s 3s/step - loss: 0.4349 - acc: 0.8411 - val_loss: 0.9766 - val_acc: 0.6824
Epoch 5/25
97/97 [==============================] - 326s 3s/step - loss: 0.3661 - acc: 0.8617 - val_loss: 1.1987 - val_acc: 0.6529
Epoch 6/25
97/97 [==============================] - 325s 3s/step - loss: 0.2681 - acc: 0.8875 - val_loss: 0.9087 - val_acc: 0.6941
Epoch 7/25
97/97 [==============================] - 325s 3s/step - loss: 0.2292 - acc: 0.9195 - val_loss: 1.0251 - val_acc: 0.6647
Epoch 8/25
97/97 [==============================] - 326s 3s/step - loss: 0.1248 - a

In [34]:
model.save('body.h5')

In [35]:
!tar -zcvf body.tgz body.h5

body.h5


In [36]:
ls -1 

body.h5
body.tgz
Dataset/


In [8]:
!pip install watson-machine-learning-client --upgrade

# Connecting with IBM CLOUD

In [4]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {"url":"https://us-south.ml.cloud.ibm.com", "apikey":"apikey"}
client = APIClient(wml_credentials)

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [5]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [1]:
space_uid = guid_from_space_name(client, 'spacename')
#space_uid

In [7]:
client.set.default_space(space_uid)

'SUCCESS'

In [2]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
#software_spec_uid

In [62]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [69]:
model_details = client.repository.store_model(model = 'body.tgz' , meta_props = {
    client.repository.ModelMetaNames.NAME : "body", 
    client.repository.ModelMetaNames.TYPE : "tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})
model_id = client.repository.get_model_id(model_details)


In [71]:
client.repository.download(model_id, 'body_cloud.tar.gz')


Successfully saved model content to file: 'body_cloud.tar.gz'


'/home/wsuser/work/body_cloud.tar.gz'

In [72]:
model_body = load_model('body.h5')

# MODEL FOR LEVEL TYPE DETECTION

In [14]:
trainPath = '/home/wsuser/work/Dataset/level/training'
testPath = '/home/wsuser/work/Dataset/level/validation'

In [15]:
training_set = train_datagen.flow_from_directory(trainPath,target_size=(244,244),batch_size=10,class_mode='categorical')
test_set = train_datagen.flow_from_directory(testPath,target_size=(244,244),batch_size=10,class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [16]:
training_set.class_indices

{'01-minor': 0, '02-moderate': 1, '03-severe': 2}

# Declaring Model Variable

In [17]:
vgg=VGG16(input_shape=(244,244,3),weights='imagenet',include_top=False)



for layer in vgg.layers:
  layer.trainable=False


x=Flatten()(vgg.output)


prediction=Dense(3,activation='softmax')(x)


model1=Model(inputs=vgg.input,outputs=prediction)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0     

In [18]:
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

# Training model

In [19]:
y = model1.fit_generator(
    training_set,
    validation_data = test_set,
    epochs = 25,
    steps_per_epoch=979//10,
    validation_steps = 171//10
)

/tmp/wsuser/ipykernel_7116/2253460449.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  y = model1.fit_generator(


Epoch 1/25
97/97 [==============================] - 328s 3s/step - loss: 1.1968 - acc: 0.5614 - val_loss: 1.0383 - val_acc: 0.6000
Epoch 2/25
97/97 [==============================] - 324s 3s/step - loss: 0.7241 - acc: 0.7265 - val_loss: 0.8500 - val_acc: 0.6471
Epoch 3/25
97/97 [==============================] - 323s 3s/step - loss: 0.5437 - acc: 0.7967 - val_loss: 0.9910 - val_acc: 0.5941
Epoch 4/25
97/97 [==============================] - 324s 3s/step - loss: 0.3847 - acc: 0.8596 - val_loss: 0.9415 - val_acc: 0.6471
Epoch 5/25
97/97 [==============================] - 323s 3s/step - loss: 0.3124 - acc: 0.8906 - val_loss: 1.1811 - val_acc: 0.5882
Epoch 6/25
97/97 [==============================] - 323s 3s/step - loss: 0.2261 - acc: 0.9112 - val_loss: 1.2515 - val_acc: 0.6294
Epoch 7/25
97/97 [==============================] - 323s 3s/step - loss: 0.1783 - acc: 0.9412 - val_loss: 1.2657 - val_acc: 0.5824
Epoch 8/25
97/97 [==============================] - 326s 3s/step - loss: 0.1439 - a

In [20]:
model1.save('level.h5')

In [21]:
!tar -zcvf level.tgz level.h5

level.h5


In [22]:
ls -1 

body_cloud.tar.gz
body.h5
body.tgz
Dataset/
level.h5
level.tgz


In [26]:
model_details = client.repository.store_model(model = 'level.tgz' , meta_props = {
    client.repository.ModelMetaNames.NAME : "level", 
    client.repository.ModelMetaNames.TYPE : "tensorflow_rt22.1",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})
model_id = client.repository.get_model_id(model_details)


In [3]:
client.repository.download(model_id, 'level_cloud.tar.gz')


In [ ]:
model_body = load_model('level.h5')

In [29]:
os.listdir('.')

['level.h5',
 '.virtual_documents',
 'level_cloud.tar.gz',
 'level.tgz',
 'body.h5',
 'body.tgz',
 'Dataset',
 'body_cloud.tar.gz']

In [4]:
client.repository.download('model_id','body_cloud.tar.gz')

In [ ]:
client.repository.download('model1_id','level_cloud.tar.gz')